In [7]:
import sys
!{sys.executable} -m pip install sentence-transformers



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chargement des données

In [3]:
profiles_df = pd.read_csv("Data/profile_model.csv").fillna("")
offers_df = pd.read_csv("Data/offres_model.csv").fillna("")

Configuration des poids par champ : Pondération des champs (selon importance pour le matching)

In [6]:
weights = {
    "Metier_regroupe": 0.20,
    "Points_forts": 0.15,
    "Compétence": 0.15,
    "Contrat": 0.15,
    "Experience_mois": 0.20,
    "Departement": 0.15
}

Correspondances entre colonnes des profils et des offres

In [7]:
field_map = {
    "Metier_regroupe": "groupe_metier",
    "Points_forts": "stack_technique",
    "Compétence": "stack_technique",
    "Contrat": "Contrat",
    "Experience_mois": "experience_mois",
    "Departement": "Departement",
}


Chargement du modèle SBERT : adapté pour les textes en français

In [8]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

Calcul des embeddings + similarités pondérées

In [9]:
# Matrice finale de similarité
similarity_total = np.zeros((len(profiles_df), len(offers_df)))

In [10]:
# Pour chaque champ 
for profile_field, weight in weights.items():
    
    offer_field = field_map[profile_field]
    profile_texts = profiles_df[profile_field].astype(str).tolist()
    offer_texts = offers_df[offer_field].astype(str).tolist()

    # Embeddings
    profile_embeddings = model.encode(profile_texts, show_progress_bar=True)
    offer_embeddings = model.encode(offer_texts, show_progress_bar=True)

    # Similarité
    sim_matrix = cosine_similarity(profile_embeddings, offer_embeddings)

    # Pondération et ajout à la matrice finale
    similarity_total += weight * sim_matrix


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches: 100%|██████████| 70/70 [00:03<00:00, 22.97it/s]


Extraction du Top 10 des paires les plus similaires

 Pour chaque profil on va garder l'offre avec le meilleur score uniquement pour eviter la redondance

In [11]:
best_matches = []
for i in range(similarity_total.shape[0]):
    best_offer_index = np.argmax(similarity_total[i])
    best_score = similarity_total[i, best_offer_index]
    best_matches.append((i, best_offer_index, best_score))

# Trier par score décroissant et prendre les top 50
top_50_unique = sorted(best_matches, key=lambda x: x[2], reverse=True)[:50]


Affichage des résultats

In [12]:
print("\nTop 10 Paires Offre/Profil (SBERT Pondéré, sans doublon profil) :\n")
for i, j, score in top_50_unique[:10]:
    print(f"Profil : {profiles_df.iloc[i]['Profil'][:60]}...")
    print(f"Offre  : {offers_df.iloc[j]['Nom_poste']}")
    print(f"Score pondéré SBERT : {round(score, 3)}")
    print("---")


Top 10 Paires Offre/Profil (SBERT Pondéré, sans doublon profil) :

Profil : Data Scientist...
Offre  : Data Scientist F/H - Système, réseaux, données (H/F)
Score pondéré SBERT : 0.888
---
Profil : Directeur  Systèmes d'Information...
Offre  : Ingénieur(e) Data Senior (H/F)
Score pondéré SBERT : 0.887
---
Profil : Journaliste sportif...
Offre  : Expert Data Center CFO CFA (H/F)
Score pondéré SBERT : 0.88
---
Profil : Directeur des Systèmes d'information (DSI)...
Offre  : Ingénieur Méthodes & Data Management F/H (H/F)
Score pondéré SBERT : 0.878
---
Profil : Data Analyst...
Offre  : Data Analyst - F/H - Système, réseaux, données (H/F)
Score pondéré SBERT : 0.874
---
Profil : DATA SCIENTIST...
Offre  : #SALONDEMANDELIEU2024 : Machine Learning Engineer  (H/F)
Score pondéré SBERT : 0.873
---
Profil : Préparateur physique, analyste de performance, formateur, sp...
Offre  : Data Owner F/H - Direction recherche et développement (H/F)
Score pondéré SBERT : 0.871
---
Profil : Motivé par l’oppor

Export des Top 50 correspondances complètes

In [ ]:
export_rows = []
for i, j, score in top_50_unique:
    profile_data = profiles_df.loc[i, ['Profil', 'Metier_regroupe', 'Points_forts', 'Compétence', 'Contrat', 'Experience_mois','Departement']].to_dict()
    offer_data = offers_df.loc[j, ['Nom_poste', 'groupe_metier', 'stack_technique', 'Contrat', 'experience_mois','Departement']].to_dict()
    row = {**profile_data, **offer_data, "Score_SBERT": round(score, 3)}
    export_rows.append(row)

result_df = pd.DataFrame(export_rows)
result_df.to_excel("Data/Top_50_Match.xlsx", index=False)

: 